In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, LongType, DoubleType, FloatType, DataType, DateType, TimestampType,DecimalType
from pyspark.sql.functions import regexp_replace,row_number
from pyspark.sql.window import Window

In [2]:
spark = (SparkSession
 .builder
 .appName("Fact")
 .getOrCreate())

22/09/08 03:23:43 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.197.132 instead (on interface ens33)
22/09/08 03:23:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/08 03:23:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
"""

Afther fetching all of the data and tranforming it, we read the data from the respective folder 

"""


profession_df = spark.read.option('header','True').csv('/home/ubuntu/Documents/Salary_Analysis/Profession/*.csv')

state_city_df = spark.read.option('header','True').csv('/home/ubuntu/Documents/Salary_Analysis/State/*.csv')

state_salary_df = spark.read.option('header','True').csv('/home/ubuntu/Documents/Salary_Analysis/State_Salary_Deviation/*.csv')



22/09/08 03:23:54 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: //home/ubuntu/Documents/Salary_Analysis/State_Salary_Deviation/*.csv.
org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "null"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:370)
	at org.apache.spark.sql.DataFrameReader.l

Py4JJavaError: An error occurred while calling o37.csv.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "null"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:747)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:745)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNecessary(Da

In [ ]:
profession_df.show()

In [ ]:
state_city_df.show()

In [ ]:
state_salary_df.show()

In [ ]:
"""

Using Spark Sql to get data in one Fact Table so that this table can be used for visualization

"""


state_salary_df.createOrReplaceTempView('state_salary_table')
state_city_df.createOrReplaceTempView('state_city_table')
profession_df.createOrReplaceTempView('profession_table')

state_city_salary = spark.sql("SELECT t1.State,t1.Salary_Deviation,t2.City FROM state_salary_table t1 INNER JOIN state_city_table t2 ON t1.State LIKE t2.State")

#fact = state_salary_df.select('State','State_Salary_Deviation','City').join(state_city_df, state_salary_df.State == state_city_df.State).show()

In [ ]:
state_city_salary.createOrReplaceTempView('state_city_salary_table')

In [ ]:


final_df = spark.sql("SELECT t1.Profession, t1.Salary, t2.*,(Salary + (Salary*Salary_Deviation)/100 ) as Sal_Dev FROM profession_table as t1 CROSS JOIN state_city_salary_table as t2")

In [ ]:
final_df.write.format('csv').option('header','true').mode('overwrite').option('encoding','UTF-8').save('Fact')